In [65]:
#autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
from askharrison.llmparse.schema_recommender import SchemaGenerator

In [67]:
from askharrison.llm.openai_llm_client import OpenAIClient

In [68]:
openai_client = OpenAIClient()

In [69]:
schema_generator = SchemaGenerator(openai_client)

In [7]:
output = schema_generator.generate_schema_from_description("generate a qa schema from chat history of a support chat channel")

In [70]:
from askharrison.llm_models import extract_python_code, safe_eval
from askharrison.llmparse.uitl import format_json_schema

In [18]:
print(format_json_schema(output))

['SupportChatHistory:', '  Type: object', '  Properties:', '    chat_id:', '      Type: string', '      Description: A unique identifier for the chat session', '    start_time:', '      Type: string', '      Description: The timestamp when the chat session began', '    end_time:', '      Type: string', '      Description: The timestamp when the chat session ended', '    participants:', '      Type: array', '      Description: List of participants in the chat', '      Array items:', '        Root:', '          Type: object', '          Properties:', '            user_id:', '              Type: string', '              Description: Unique identifier for the user', '            role:', '              Type: string', '              Description: Role of the participant in the chat', "              Constraints: allowed values: ['agent', 'customer']", '            name:', '              Type: string', '              Description: Name of the participant', '          Required fields: user_id, rol

In [9]:
import json


In [26]:
output2 = schema_generator.generate_schema_from_description("""generate a qa schema from chat history of a support chat channel
                                                        The schema should have a field for the question and a field for the answer or resolution
                                                        answer is null if the question is not answered
                                                        also include start_date and end_date""")


In [28]:
output2

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'title': 'Support Chat QA',
 'type': 'object',
 'properties': {'question': {'type': 'string',
   'description': 'The question asked in the support chat.'},
  'answer': {'type': ['string', 'null'],
   'description': 'The answer or resolution provided in the support chat, or null if the question was unanswered.'},
  'start_date': {'type': 'string',
   'format': 'date-time',
   'description': 'The date and time when the question was asked or the chat session began.'},
  'end_date': {'type': ['string', 'null'],
   'format': 'date-time',
   'description': 'The date and time when the chat session ended, or null if the chat is ongoing.'},
  'chat_id': {'type': 'string',
   'description': 'A unique identifier for the chat session.'},
  'user_id': {'type': 'string',
   'description': 'A unique identifier for the user asking the question.'},
  'agent_id': {'type': ['string', 'null'],
   'description': 'A unique identifier for the support ag

In [32]:
output_with_feedback = schema_generator.update_schema_from_feedback(output2, "remove agent_id")

In [33]:
output_with_feedback 

{'description': "An updated schema for the Support Chat QA with improvements and removal of the 'agent_id' field.",
 'schema': {'$schema': 'http://json-schema.org/draft-07/schema#',
  'title': 'Support Chat QA',
  'type': 'object',
  'properties': {'question': {'type': 'string',
    'description': 'The question asked in the support chat.'},
   'answer': {'type': ['string', 'null'],
    'description': 'The answer or resolution provided in the support chat, or null if the question was unanswered.'},
   'start_date': {'type': 'string',
    'format': 'date-time',
    'description': 'The date and time when the question was asked or the chat session began.'},
   'end_date': {'type': ['string', 'null'],
    'format': 'date-time',
    'description': 'The date and time when the chat session ended, or null if the chat is ongoing.'},
   'chat_id': {'type': 'string',
    'description': 'A unique identifier for the chat session.'},
   'user_id': {'type': 'string',
    'description': 'A unique ident

In [71]:
with open("C:\\Users\\alist\\Desktop\\code\\askharrison\\notebooks\\data\\cc_features.txt") as f:
    text = f.read()

In [79]:
schema_generator.clear_message_history()
recommended_output = schema_generator.recommend_schema_from_document(text)

In [83]:
from askharrison.llm_models import process_question

In [84]:
test_outputs = process_question(f"""generate example output for the schema: {recommended_output}""")

In [86]:
safe_eval(extract_python_code(test_outputs))

{'updates': [{'date': '2023-10-15',
   'title': 'Enhanced Reporting Dashboards',
   'description': 'We have improved the reporting dashboard with new visualizations and real-time data monitoring capabilities to enhance decision-making processes.',
   'section': "What's new",
   'additionalInfo': {'links': ['https://www.webex.com/contact-center/reporting-dashboard-upgrade',
     'https://www.webex.com/news/2023-october-update'],
    'features': ['Real-time data monitoring',
     'Advanced visualizations',
     'Customizable dashboard layout']}},
  {'date': '2023-11-01',
   'title': 'Upcoming AI-Powered Customer Insights',
   'description': 'We are excited to announce that AI-powered customer insights will be available soon. This includes sentiment analysis and customer journey mapping to better understand customer needs.',
   'section': 'Coming soon',
   'additionalInfo': {'links': ['https://www.webex.com/contact-center/ai-insights-preview'],
    'features': ['Sentiment analysis', 'Cust

In [89]:
from askharrison.llmparse.document_parser import DocumentParser
document_parser = DocumentParser(openai_client) 

In [92]:
release_document_output = document_parser.parse_document(text, recommended_output)

Document too large, splitting into chunks
Document too large, splitting into 2 chunks
Fields description:
{'description': 'JSON schema for parsing Webex Contact Center update announcements', 'type': 'object', 'properties': {'updates': {'type': 'array', 'items': {'type': 'object', 'properties': {'date': {'type': 'string', 'format': 'date', 'description': 'The date the update was announced.'}, 'title': {'type': 'string', 'description': 'The title of the update or feature.'}, 'description': {'type': 'string', 'description': 'A detailed description of the update or feature.'}, 'section': {'type': 'string', 'enum': ["What's new", 'Coming soon', 'Limitations', 'Announcements'], 'description': 'Section of the update in the document.'}, 'additionalInfo': {'type': 'object', 'properties': {'links': {'type': 'array', 'items': {'type': 'string', 'format': 'uri', 'description': 'Links to more details or documentation.'}}, 'features': {'type': 'array', 'items': {'type': 'string', 'description': 'Bri

In [91]:
release_document_output

[{'updates': [{'date': '2025-01-27',
    'title': 'Introducing cookies consent for Livechat widget',
    'description': "Businesses can ensure they respect their customers' privacy choices by allowing them to explicitly consent to cookies stored by our Livechat service before starting a chat session. Additionally, we now offer an option to link a business's privacy URL to the widget so that customers can understand how their data is tracked and used by the business. This optional configuration will not impact existing widgets.",
    'section': "What's new",
    'additionalInfo': {'links': [], 'features': []}},
   {'date': '2025-01-27',
    'title': 'Retain email addresses in the To field',
    'description': 'Agents can now add multiple email addresses in the To field when replying to all in a conversation. Additionally, agents can remove any email address from the list except the primary one.',
    'section': "What's new",
    'additionalInfo': {'links': [], 'features': []}},
   {'dat

In [42]:
safe_eval(extract_python_code(release_document_output))

{'title': "What's new for administrators in Webex Contact Center",
 'intro': 'We want to make sure that you know about the major updates made available in Webex Contact Center for administrators—updates to the interface, new functionality, and ways to manage your contact center.',
 'newFeatures': [{'date': 'January 27, 2025',
   'feature': 'Introducing cookies consent for Livechat widget',
   'description': "Businesses can ensure they respect their customers' privacy choices by allowing them to explicitly consent to cookies stored by our Livechat service before starting a chat session."},
  {'date': 'January 27, 2025',
   'feature': 'Retain email addresses in the To field',
   'description': 'Agents can now add multiple email addresses in the To field when replying to all in a conversation.'},
  {'date': 'January 24, 2025',
   'feature': 'Agent initiated outbound SMS and Email support',
   'description': 'Supervisors can initiate an outbound SMS or Email Task from the Webex Contact Cen

In [48]:
with open("C:\\Users\\alist\\Desktop\\code\\askharrison\\askharrison\\llm_models.py") as f:
    codefile = f.read()
    

In [49]:
code_recommended_output = schema_generator.recommend_schema_from_document(codefile)

In [50]:
code_recommended_output

{'description': 'A set of Python functions for language model interaction and utility tasks, including data processing, concurrency, and input chunking.',
 'schema': {'imports': {'type': 'array',
   'items': {'type': 'string'},
   'description': 'List of imported libraries and modules.'},
  'functions': {'type': 'object',
   'properties': {'process_question': {'type': 'object',
     'description': 'Process a question using the specified language model.',
     'properties': {'input': {'type': 'object',
       'properties': {'question': {'type': 'string',
         'description': 'The question to be processed by the language model.'},
        'model': {'type': 'string',
         'description': 'The model used for processing the question.'}}},
      'output': {'type': 'string',
       'description': 'Response generated by the language model.'}}},
    'polish_code': {'type': 'object',
     'description': 'Polishes code by removing unwanted characters.',
     'properties': {'input': {'type':

In [64]:
codefile_schama_dict = json.loads(extract_python_code(code_recommended_output))

# create a pydantic model from the schema without using the schema generator
from pydantic import BaseModel
     

In [68]:
pydantic_output = openai_client.generate(f"""generate a pydantic models from the schema below
                       {codefile_schama_dict}"
                       return only python code for the pydantic model,
                       example output:
                       ####
                       ```python
                       class MyModel(BaseModel):
                            field1: str
                            field2: int
                       
                       class MyModel2(BaseModel):
                            field1: MyModel
                            field2: int
                       ```
                       ####
                       output
                       """)

In [74]:
print(extract_python_code(pydantic_output))

from typing import List, Optional
from pydantic import BaseModel

class Parameter(BaseModel):
    name: str
    type: str
    description: str
    default: Optional[str] = None

class Function(BaseModel):
    name: str
    description: str
    parameters: List[Parameter]
    returns: str

class Schema(BaseModel):
    imports: List[str]
    functions: List[Function]


In [79]:
from askharrison.llmparse.document_parser import DocumentParser
document_parser = DocumentParser(openai_client) 

In [80]:
document_parser = DocumentParser(openai_client) 

In [81]:
doc_parsed_output = document_parser.parse_document(codefile, codefile_schama_dict)

Invalid JSON in LLM response, returning raw response


In [1]:
safe_eval(extract_python_code(doc_parsed_output))

NameError: name 'safe_eval' is not defined

In [10]:
from openai import OpenAI

# get the text with https://help.webex.com/en-us/article/8dmbcr/What's-New-in-Webex-Suite
import requests
url = "https://help.webex.com/en-us/article/8dmbcr/What's-New-in-Webex-Suite"

response = requests.get(url)
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser')
text = soup.get_text()

In [11]:
#export text to file
with open("C:\\Users\\alist\\Desktop\\code\\askharrison\\notebooks\\data\\webex_new.txt", "w") as f:
    f.write(text)

In [58]:
from askharrison.llmparse.document_parser import DocumentParser
document_parser = DocumentParser(openai_client) 

In [21]:
recommended_schema = schema_generator.recommend_schema_from_document(text)

In [41]:
recommended_schema2 = schema_generator.update_schema_from_feedback(recommended_schema, "make it simpler")

In [53]:
with open("C:\\Users\\alist\\Desktop\\code\\askharrison\\notebooks\\data\\release_schema.json", "w") as f:
    json.dump(recommended_schema2, f)

In [42]:
from askharrison.llmparse.uitl import format_json_schema
format_json_schema(recommended_schema2)

['Root:',
 '  Type: object',
 '  Description: A simplified schema for parsing Webex update logs.',
 '  Properties:',
 '    updates:',
 '      Type: array',
 '      Array items:',
 '        Root:',
 '          Type: object',
 '          Properties:',
 '            date:',
 '              Type: string',
 '              Description: Date of the update.',
 '            version:',
 '              Type: string',
 '              Description: Version number of the release.',
 '            platforms:',
 '              Type: array',
 '              Array items:',
 '                Root:',
 '                  Type: string',
 '                  Description: Platform(s) affected (e.g., Windows, Mac).',
 '            changes:',
 '              Type: string',
 '              Description: Summary of new features, fixes, and changes.',
 '            announcements:',
 '              Type: array',
 '              Array items:',
 '                Root:',
 '                  Type: string',
 '              

In [64]:
from askharrison.llm.token_util import clip_text_by_token

text_truncated = clip_text_by_token(text, 10000)

# export text_truncated to file
with open("C:\\Users\\alist\\Desktop\\code\\askharrison\\notebooks\\data\\webex_new_truncated.txt", "w") as f:
    f.write(text_truncated)

In [59]:
doc_parsed_output = document_parser.parse_document(text_truncated, recommended_schema)

Document too large, splitting into chunks
Document too large, splitting into 3 chunks
Invalid JSON in LLM response, returning raw response
Invalid JSON in LLM response, returning raw response
Invalid JSON in LLM response, returning raw response


In [63]:
# get type of each doc_parsed_output

doc_parsed_output_types = [type(x) for x in doc_parsed_output]
doc_parsed_update_types = [type(x.get("updates")) for x in doc_parsed_output]
doc_parsed_update_types 

doc_parsed_output_updates = [x.get("updates") for x in doc_parsed_output]
len(doc_parsed_output_updates)

3

In [62]:
doc_parsed_output_updates[2]

[{'date': 'November 1, 2023',
  'update': 'End of support for Android 9 in Webex App',
  'details': 'Google stopped releasing new security updates for Android 9, ending support for Android 9 in Webex starting November 1, 2023.'},
 {'date': 'October 4, 2023',
  'update': 'Article renamed',
  'details': "Renamed article from 'Webex App | What's New' to 'What's New in Webex Suite'."},
 {'date': 'October 4, 2023',
  'update': 'Updated app versions',
  'details': 'New app versions for various platforms: Windows—43.10.0.27451, Mac—43.10.0.27451, iPhone and iPad—43.10, Android—43.10.'},
 {'date': 'September 29, 2023',
  'update': 'Support for iOS 17 and iPadOS 17',
  'details': 'Webex App is supported on iOS 17 and iPadOS 17.'},
 {'date': 'September 28, 2023',
  'update': 'Support for macOS Sonoma 14.0',
  'details': 'Announced Day 0 support for macOS Sonoma 14.0.'},
 {'date': 'September 15, 2023',
  'update': 'End of life announcement for Unicon eLux RP6 2104 LTSR and Ubuntu 16.04 LTS',
  'd